In [ ]:
pip install scapy

In [ ]:
import socket
from scapy.all import *

### Port Scan

In [ ]:
def scan_ports(ip):
    open_ports = []
    for port in range(1, 1025):
        pkt = IP(dst=ip)/TCP(dport=port, flags="S")
        response = sr1(pkt, timeout=1, verbose=0)
        if response:
            if response.haslayer(TCP) and response[TCP].flags == 18:
                open_ports.append(port)
                print(f"Port {port} is open.")
    return open_ports

In [ ]:
target_host = input("IP: ")
open_ports = scan_ports(target_host)

IP: 218.50.136.184
Port 22 is open.
Port 25 is open.
Port 80 is open.
Port 443 is open.


### Detect service Banner

In [ ]:
def service_banner(ip, port):
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(2)
        s.connect((ip, port))
        banner = s.recv(1024).decode('utf-8').strip()
        s.close()
        return banner
    except:
        return None

In [ ]:
def detect_service_by_banner(banner):
    if not banner:
        return "Unknown"

    service_patterns = {
        "HTTP": ["80", "HTTP/1.", "HTTP/2."],
        "SSH": ["SSH-2.0-", "SSH-1.99-"],
        "SMTP" : ["25", "ESMTP"],
        "FTP": ["220", "FTP"],
        "SQL": ["MSSQL", "MySQL", "PostgreSQL"],
    }

    for service, patterns in service_patterns.items():
        for pattern in patterns:
            if pattern in banner:
                return service

In [ ]:
for port in open_ports:
    banner = service_banner(target_host, port)
    if banner:
        service = detect_service_by_banner(banner)
        print(f"{port} banner : {banner}")
        print(f"Service: {service}")
    else:
        print(f"{port} port no banner")

22 banner : SSH-2.0-OpenSSH_8.9p1 Ubuntu-3ubuntu0.3
Service: SSH
25 banner : 220 sakuya-izyoi.skbroadband ESMTP Postfix (Ubuntu)
Service: SMTP
80 port no banner
443 port no banner


### HTTP / HTTPS

In [ ]:
def detect_http_service(ip, port):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(2)
    s.connect((ip, port))

    http_request = "GET / HTTP/1.1\r\nHost: {}\r\n\r\n".format(ip)
    s.sendall(http_request.encode())

    response = s.recv(4096).decode('utf-8')

    if "HTTP/1." in response:
        return "True"
    else:
        return "False"

In [ ]:
for port in open_ports:
  is_http = detect_http_service(target_host, port)

  if is_http:
      print(f"IP : {target_host} Port : {port} is {is_http}.")
  else:
      print(f"IP : {target_host} Port : {port} is {is_http}")

IP : 218.50.136.184 Port : 22 is False.
IP : 218.50.136.184 Port : 25 is False.
IP : 218.50.136.184 Port : 80 is True.
IP : 218.50.136.184 Port : 443 is True.


### DHCP


In [ ]:
def detect_dhcp_service(ip, port, iface="eth0"):
    dhcp_discover = (
        Ether(dst="ff:ff:ff:ff:ff:ff") /
        IP(src="0.0.0.0", dst=ip) /
        UDP(sport=68, dport=port) /
        BOOTP(chaddr=b"\x00\x01\x02\x03\x04\x05") /
        DHCP(options=[("message-type", "discover"), "end"])
    )

    response = srp1(dhcp_discover, timeout=2, verbose=0, iface=iface)

    if response and DHCP in response and response[DHCP].options[0][1] == 2:
        return True
    return False

In [ ]:
for port in open_ports:
  is_dhcp = detect_dhcp_service(target_host, port)

  if is_dhcp:
      print(f"IP : {target_host} Port : {port} is {is_dhcp}.")
  else:
      print(f"IP : {target_host} Port : {port} is {is_dhcp}")

IP : 218.50.136.184 Port : 22 is False
IP : 218.50.136.184 Port : 25 is False
IP : 218.50.136.184 Port : 80 is False
IP : 218.50.136.184 Port : 443 is False
